## Installation (work on colab)

In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
!pip install gym

!pip install free-mujoco-py
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install imageio==2.4.1
!pip install -U colabgymrender
!pip install mujoco

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.8).
The following additional packages will be installed:
  libegl-dev libgl-dev libgles-dev libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev
  libglvnd-dev libglx-dev libopengl-dev libosmesa6
The following NEW packages will be installed:
  libegl-dev libgl-dev libgl1-mesa-dev libgl1-mesa-glx libgles-dev libgles1 libglew-dev
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libopengl-dev libosmesa6
  libosmesa6-dev
0 upgraded, 15 newly installed, 0 to remove and 15 not upgraded.
Need to get 3,952 kB of archives.
After this operation, 18.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglx-dev amd64 1.4.0-1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgl-dev amd64 1.4.0-1 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [3]:
!python --version

Python 3.10.12


In [4]:
!pip install pfrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pfrl: filename=pfrl-0.4.0-py3-none-any.whl size=155460 sha256=5dc1024aa1d85d5de1e3b855a59a858797d502fc142964cd3c5b5a8d2f6141dd
  Stored in directory: /root/.cache/pip/wheels/22/4a/0f/a87cd1ae925086eb3a1b8759f620fcf48e47939fb082946c3b
Successfully built pfrl


In [5]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.3 MB/s eta 0:00:00


In [6]:
import functools
import numpy as np

import gymnasium as gym
import gym.wrappers

import pfrl

In [7]:
pip freeze > requirements.txt

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
device = "cuda:0"

In [9]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/drive


/


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


bin/    cuda@     cuda-11.8/  games/               include/  lib64/      man@   share/
colab/  cuda-11@  etc/        _gcs_config_ops.so/  lib/      licensing/  sbin/  src/


cp: cannot create regular file '/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/assets/envs/': Not a directory


In [10]:
%cd /content/drive/MyDrive/codes

/content/drive/.shortcut-targets-by-id/1f55zVGF_dF_ofwI_JfGgeIj4XSrHgcE8/codes


## Initialize environment

In [11]:
env_name = "Hopper-v3"
num_envs = 5
seed = 42

monitor = False
render = False

process_seeds = np.arange(num_envs) + seed * num_envs

In [12]:
def make_env(process_idx, test):
    env = gym.make(env_name, xml_file="envs/sandpaper_hopper.xml")
    # Unwrap TimiLimit wrapper
    assert isinstance(env, gym.wrappers.TimeLimit)
    env = env.env
    # Use different random seeds for train and test envs
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32 - 1 - process_seed if test else process_seed
    env.seed(env_seed)
    # Cast observations to float32 because our model uses float32
    env = pfrl.wrappers.CastObservationToFloat32(env)
    # Normalize action space to [-1, 1]^n
    env = pfrl.wrappers.NormalizeActionSpace(env)
    if monitor:
        env = gym.wrappers.Monitor(env, "monitor")
    if render:
        env = pfrl.wrappers.Render(env)
    return env

In [13]:
def make_batch_env(test):
    return pfrl.envs.MultiprocessVectorEnv(
        [
            functools.partial(make_env, idx, test)
            for idx, env in enumerate(range(num_envs))
        ]
    )

In [14]:
env = make_env(0, test=False)

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


Compiling /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.10/dist-packages/mujoco_py/cymj.pyx


INFO:root:running build_ext
INFO:root:building 'mujoco_py.cymj' extension
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-310/usr/local/lib
INFO:root:creating /usr/local/lib/python3.10/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_linuxcpuextensionbuilder/temp.linux-x86_64-cpython-31

In [15]:
env.observation_space

Box(-inf, inf, (11,), float64)

In [16]:
env.action_space

Box(-1.0, 1.0, (3,), float32)

In [17]:
obs_size = env.observation_space.shape[0]
action_size = env.action_space.shape[0]
timestep_limit = env.spec.max_episode_steps

## SAC Config (with PFRL)

sample codes and parameters in [this file](https://github.com/pfnet/pfrl/blob/master/examples/mujoco/reproduction/soft_actor_critic/train_soft_actor_critic.py).

In [18]:
import torch
from torch import distributions, nn

from pfrl import experiments, replay_buffers, utils
from pfrl.nn.lmbda import Lambda

In [19]:
# default params
policy_output_scale = 1.0

policy_hidden_dim = 256
policy_lr = 3e-4

q_func_hidden_dim = 256
q_func_lr = 3e-4

temperature_optimizer_lr = 3e-4

buffer_size = 10 ** 6
replay_start_size = 10 ** 4
batch_size = 256
gamma = 0.99

# steps = 10 ** 6
steps = 10 * 10 ** 5

In [20]:
def burnin_action_func():
    """Select random actions until model is updated one or more times."""
    return np.random.uniform(env.action_space.low, env.action_space.high).astype(np.float32)

In [21]:
def squashed_diagonal_gaussian_head(x):
    assert x.shape[-1] == action_size * 2
    mean, log_scale = torch.chunk(x, 2, dim=1)
    log_scale = torch.clamp(log_scale, -20.0, 2.0)
    var = torch.exp(log_scale * 2)
    base_distribution = distributions.Independent(
        distributions.Normal(loc=mean, scale=torch.sqrt(var)), 1
    )
    # cache_size=1 is required for numerical stability
    return distributions.transformed_distribution.TransformedDistribution(
        base_distribution, [distributions.transforms.TanhTransform(cache_size=1)]
    )

In [22]:
def make_policy_with_optimizer():
    policy = nn.Sequential(
        nn.Linear(obs_size, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, policy_hidden_dim),
        nn.ReLU(),
        nn.Linear(policy_hidden_dim, action_size * 2),
        Lambda(squashed_diagonal_gaussian_head),
    )
    torch.nn.init.xavier_uniform_(policy[0].weight)
    torch.nn.init.xavier_uniform_(policy[2].weight)
    torch.nn.init.xavier_uniform_(policy[4].weight, gain=policy_output_scale)
    policy_optimizer = torch.optim.Adam(policy.parameters(), lr=policy_lr)
    return policy, policy_optimizer

In [23]:
def make_q_func_with_optimizer():
    q_func = nn.Sequential(
        pfrl.nn.ConcatObsAndAction(),
        nn.Linear(obs_size + action_size, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, q_func_hidden_dim),
        nn.ReLU(),
        nn.Linear(q_func_hidden_dim, 1),
    )
    torch.nn.init.xavier_uniform_(q_func[1].weight)
    torch.nn.init.xavier_uniform_(q_func[3].weight)
    torch.nn.init.xavier_uniform_(q_func[5].weight)
    q_func_optimizer = torch.optim.Adam(q_func.parameters(), lr=q_func_lr)
    return q_func, q_func_optimizer

In [24]:
policy, policy_optimizer = make_policy_with_optimizer()
q_func1, q_func1_optimizer = make_q_func_with_optimizer()
q_func2, q_func2_optimizer = make_q_func_with_optimizer()

In [25]:
rbuffer = replay_buffers.ReplayBuffer(buffer_size)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Train SAC policy online

In [26]:
agent = pfrl.agents.SoftActorCritic(
    policy,
    q_func1,
    q_func2,
    policy_optimizer,
    q_func1_optimizer,
    q_func2_optimizer,
    rbuffer,
    gamma=gamma,
    replay_start_size=replay_start_size,
    gpu=0 if torch.cuda.is_available() else -1,
    minibatch_size=batch_size,
    burnin_action_func=burnin_action_func,
    entropy_target=-action_size,
    temperature_optimizer_lr=temperature_optimizer_lr,
)


In [ ]:
experiments.train_agent_batch_with_evaluation(
    agent=agent,
    env=make_batch_env(test=False),
    eval_env=make_batch_env(test=False),
    outdir="logs/trying_something_out",
    steps=steps,
    eval_n_steps=None,
    eval_n_episodes=10,
    eval_interval=5 * 10 ** 3,
    log_interval=10 ** 3,
    max_episode_len=timestep_limit,
)
agent.save("sac-default")

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment Hopper-v3 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/envs/mujoco/mujoco_env.py:237: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may

In [ ]:
agent.save("sac-default")

## online evaluation

In [ ]:
agent.load("sac-default")

In [ ]:
eval_stats = experiments.eval_performance(
    env=make_batch_env(test=True),
    agent=agent,
    n_steps=None,
    n_episodes=10,
    max_episode_len=timestep_limit,
)
print(
    "n_runs: {} mean: {} median: {} stdev {}".format(
        10,
        eval_stats["mean"],
        eval_stats["median"],
        eval_stats["stdev"],
    )
)

## render (TODO)

In [ ]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
def make_env_for_render():
    env = gym.make(env_name, render_mode='rgb_array')
    env = gym.wrappers.RecordEpisodeStatistics(env)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env

In [ ]:
env_ = make_env_for_render()

In [ ]:
plt.axis('off')
done = False
obs = env_.reset()

i = 0
while not done:
    i += 1
    if i % 20 == 0:
        ipythondisplay.clear_output(wait=True)
        # print("At timestep = ", i)
        screen = env_.render()
        plt.imshow(screen[0])
        ipythondisplay.display(plt.gcf())

    action = agent.act(obs)
    obs, reward, done, info = env_.step(action)

    if done:
        break